In [ ]:
%pip install gdown
import gdown

# Replace with your file's unique ID
file_id = "1lhAaeQCmk2y440PmagA0KmIVBIysVMwu"
gdown.download(f"https://drive.google.com/uc?id={file_id}", "tennis_court_det_dataset.zip", quiet=False)

In [ ]:
!unzip tennis_court_det_dataset.zip

In [17]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms

import json
import cv2
import numpy as np

In [25]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


# Create Torch Dataset

In [19]:
class KeypointsDataset(Dataset):
    def __init__(self, img_dir, data_file):
        self.img_dir = img_dir
        with open(data_file, "r") as f:
            self.data = json.load(f)
        
        self.transforms = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        item = self.data[idx]
        img = cv2.imread(f"{self.img_dir}/{item['id']}.png")
        h,w = img.shape[:2]

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = self.transforms(img)
        kps = np.array(item['kps']).flatten()
        kps = kps.astype(np.float32)

        kps[::2] *= 224.0 / w # Adjust x coordinates
        kps[1::2] *= 224.0 / h # Adjust y coordinates

        return img, kps

In [20]:
train_dataset = KeypointsDataset("data/images","data/data_train.json")
val_dataset = KeypointsDataset("data/images","data/data_val.json")

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)

# Create model

In [21]:
model = models.resnet50(pretrained=True)
model.fc =  torch.nn.Linear(model.fc.in_features, 14*2) # Replaces the last layer

/home/montek/coding/tennis_anaylsis/venv/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/montek/coding/tennis_anaylsis/venv/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /home/montek/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:02<00:00, 40.6MB/s]


In [22]:
model = model.to(device)

# Train

In [23]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [26]:
epochs=20
for epoch in range(epochs):
    for i, (imgs,kps) in enumerate(train_loader):
        imgs = imgs.to(device)
        kps = kps.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, kps)
        loss.backward()
        optimizer.step()

        if i % 10 == 0:
            print(f"Epoch {epoch}, iter {i}, loss: {loss.item()}")

Epoch 0, iter 0, loss: 1197.982421875
Epoch 0, iter 10, loss: 1281.4176025390625
Epoch 0, iter 20, loss: 993.949951171875
Epoch 0, iter 30, loss: 1081.052490234375
Epoch 0, iter 40, loss: 819.0162353515625
Epoch 0, iter 50, loss: 1102.30419921875
Epoch 0, iter 60, loss: 876.8107299804688
Epoch 0, iter 70, loss: 836.4522094726562
Epoch 0, iter 80, loss: 697.8380737304688
Epoch 0, iter 90, loss: 718.424560546875
Epoch 0, iter 100, loss: 615.0025634765625
Epoch 0, iter 110, loss: 630.0537109375
Epoch 0, iter 120, loss: 564.9267578125
Epoch 0, iter 130, loss: 462.025390625
Epoch 0, iter 140, loss: 499.7392883300781
Epoch 0, iter 150, loss: 624.5834350585938
Epoch 0, iter 160, loss: 356.9648742675781
Epoch 0, iter 170, loss: 435.9482727050781
Epoch 0, iter 180, loss: 405.6845397949219
Epoch 0, iter 190, loss: 286.9084167480469
Epoch 0, iter 200, loss: 308.7380065917969
Epoch 0, iter 210, loss: 306.2906494140625
Epoch 0, iter 220, loss: 232.95306396484375
Epoch 0, iter 230, loss: 323.0076293

In [ ]:
torch.save(model.state_dict(), "keypoints_model.pth")